# Coding Attention Mechanism

## Simple Self-Attention Without Trainable Weights

Let's create a simple example:

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

inputs.shape # [token, embedding_dim]

torch.Size([6, 3])

In [4]:
query_2 = inputs[1] # 2nd input token is the query
attention_scores_2 = torch.empty(inputs.shape[0])

# compute attention scores
for i, input_embedding in enumerate(inputs):
  attention_scores_2[i] = torch.dot(query_2, input_embedding)

attention_scores_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [5]:
# compute attention weights
attention_weights_2 = torch.softmax(attention_scores_2, dim=0)

print("attention weights: ", attention_weights_2)
print("sum:", attention_weights_2.sum())

attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
sum: tensor(1.)


In [7]:
# compute context vector
context_vector_2 = torch.zeros(inputs.shape[1])

for i, input_embedding in enumerate(inputs):
  context_vector_2 += attention_weights_2[i] * input_embedding

context_vector_2

tensor([0.4419, 0.6515, 0.5683])

Let's compute for all queries simultaneously using matrices:

In [8]:
# compute attention scores
attention_scores = inputs @ inputs.T
attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [9]:
# compute attention weights
attention_weights = torch.softmax(attention_scores, dim=-1)
print("row sums:", attention_weights.sum(dim=-1))
attention_weights

row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [10]:
# compute context vectors
context_vectors = attention_weights @ inputs
context_vectors

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## Implement Self-Attention with Trainable Weights

## Compute KVQ vectors


In [29]:
# define some variables
x_2 = inputs[1] # 2nd input token is the query
input_embedding_dim = inputs.shape[1] # dim=3
output_embedding_dim = 5 # output dim=2

Let's create KVQ matrices:

In [30]:
torch.manual_seed(211)
W_query = torch.nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim),
                             requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim),
                           requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim),
                             requires_grad=False)

In [31]:
# project x_2 into KVQ spaces
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print("query_2:", query_2)
print("key_2:", key_2)
print("value_2:", value_2)

query_2: tensor([0.3530, 0.8074, 0.7720, 0.9246, 0.7118])
key_2: tensor([0.6885, 0.4568, 1.4283, 1.4383, 1.2989])
value_2: tensor([1.2123, 0.6055, 1.0735, 1.3861, 1.5435])


Let's compute all the projected keys and values:

In [32]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys:", keys)
print("values:", values)

keys: tensor([[0.4207, 0.2279, 1.2477, 0.9477, 0.7168],
        [0.6885, 0.4568, 1.4283, 1.4383, 1.2989],
        [0.6925, 0.4572, 1.4219, 1.4353, 1.2789],
        [0.3541, 0.2496, 0.7168, 0.7615, 0.7592],
        [0.5700, 0.3358, 0.9077, 0.9768, 0.5587],
        [0.3412, 0.2621, 0.8338, 0.8443, 0.9977]])
values: tensor([[0.6973, 0.4836, 1.1020, 1.0985, 0.9538],
        [1.2123, 0.6055, 1.0735, 1.3861, 1.5435],
        [1.2069, 0.6000, 1.0771, 1.3826, 1.5277],
        [0.6661, 0.3190, 0.4886, 0.7061, 0.8630],
        [0.7701, 0.3332, 0.8395, 0.9301, 0.8138],
        [0.7621, 0.3945, 0.4978, 0.7934, 1.0709]])


In [33]:
# compute the attention scores of x_2
attention_scores_2 = query_2 @ keys.T
attention_scores_2

tensor([2.6821, 3.9688, 3.9486, 2.1243, 2.4738, 2.4665])

In [34]:
# compute attention weights
keys_embedding_dim = keys.shape[-1]
attention_weights_2 = torch.softmax(attention_scores_2 / keys_embedding_dim**0.5, dim=-1)
attention_weights_2

tensor([0.1401, 0.2491, 0.2468, 0.1092, 0.1276, 0.1272])

In [35]:
# compute context vectors
context_vector_2 = attention_weights_2 @ values
context_vector_2

tensor([0.9655, 0.4942, 0.9114, 1.1371, 1.2295])

## Implement a Self-Attention Class

In [42]:
import torch.nn as nn

class SelfAttentionV1(nn.Module):
  def __init__(self, input_embedding_dim, output_embedding_dim):
    super().__init__()
    self.W_query = nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim))
    self.W_key = nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim))
    self.W_value = nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim))

  def forward(self, inputs):
    keys = inputs @ self.W_key
    values = inputs @ self.W_value
    queries = inputs @ self.W_query

    attention_scores = queries @ keys.T
    attention_weights = torch.softmax(
        attention_scores / keys.shape[-1]**0.5, dim=-1)
    context_vectors = attention_weights @ values
    return context_vectors

In [43]:
torch.manual_seed(211)
sa_v1 = SelfAttentionV1(input_embedding_dim, output_embedding_dim)
context_vectors = sa_v1(inputs)
context_vectors

tensor([[0.9414, 0.4835, 0.8960, 1.1131, 1.1991],
        [0.9655, 0.4942, 0.9114, 1.1371, 1.2295],
        [0.9641, 0.4936, 0.9103, 1.1357, 1.2278],
        [0.9298, 0.4771, 0.8825, 1.0980, 1.1843],
        [0.9173, 0.4713, 0.8726, 1.0845, 1.1688],
        [0.9458, 0.4847, 0.8955, 1.1156, 1.2045]], grad_fn=<MmBackward0>)

## ## Implement a Self-Attention Class With Linear Layers

In [46]:
class SelfAttentionV2(nn.Module):
  def __init__(self, input_embedding_dim, output_embedding_dim, qkv_bias=False):
    super().__init__()
    self.W_query = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)
    self.W_key = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)
    self.W_value = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)

  def forward(self, inputs):
    keys = self.W_key(inputs)
    values = self.W_value(inputs)
    queries = self.W_query(inputs)

    attention_scores = queries @ keys.T
    attention_weights = torch.softmax(
        attention_scores / keys.shape[-1]**0.5, dim=-1)
    context_vectors = attention_weights @ values
    return context_vectors

In [47]:
torch.manual_seed(211)
sa_v2 = SelfAttentionV2(input_embedding_dim, output_embedding_dim)
context_vectors = sa_v2(inputs)
context_vectors

tensor([[ 0.2783,  0.4747, -0.4724, -0.0341,  0.3074],
        [ 0.2768,  0.4746, -0.4740, -0.0356,  0.3078],
        [ 0.2768,  0.4747, -0.4741, -0.0353,  0.3078],
        [ 0.2798,  0.4775, -0.4724, -0.0288,  0.3086],
        [ 0.2779,  0.4780, -0.4733, -0.0269,  0.3090],
        [ 0.2799,  0.4764, -0.4724, -0.0316,  0.3081]], grad_fn=<MmBackward0>)